## Attention Mechanisms
1) Reasons for using attention in neural networks
2) Basic self-attention and also self-attention
3) Implementing Causal attention that allows LLM's to generate one token at a time
4) Masking random selected attention weights with dropout to reduce overfitting
5) Stacking multiple causal attention into a Multi-head attention module

In [1]:
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89],
    [0.55, 0.87, 0.66],
    [0.57, 0.85, 0.64],
    [0.22, 0.58, 0.33],
    [0.77, 0.25, 0.10],
    [0.05, 0.80, 0.55]]
)

In [2]:
query = inputs[1]
atten_scores2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    atten_scores2[i] = torch.dot(x_i, query)
print(atten_scores2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
# normalizing the attention weights so they sum to 1
atten_weights_2_tmp = atten_scores2 / atten_scores2.sum()
print("attention weights: ", atten_weights_2_tmp)
print(atten_weights_2_tmp.sum())

attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


In [4]:
# using softmax to do normalization
def softmax_naive(x):
    return torch.exp(x) / (torch.exp(x).sum(dim=0))

atten_weights_2_naive = softmax_naive(atten_scores2)
print("Attention weights naive: ", atten_weights_2_naive)
print(atten_weights_2_naive.sum())

Attention weights naive:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [5]:
# using the inbuilt torch.softmax
atten_weights_2 = torch.softmax(atten_scores2, dim=0)
print("Attention weights torch.softmax: ", atten_weights_2)
print(atten_weights_2.sum())

Attention weights torch.softmax:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [6]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += atten_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [7]:
attention_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attention_scores[i,j] = torch.dot(x_i, x_j)

print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [8]:
attention_scores_2 = inputs @ inputs.T
print(attention_scores_2)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [9]:
attention_weights = torch.softmax(attention_scores_2, dim=1)
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [10]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 Sum: ", row_2_sum)
print("ALL Rows sums: ", attention_weights.sum(dim=1))

Row 2 Sum:  1.0
ALL Rows sums:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [11]:
all_context_vecs = attention_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


### Computing weighted attention

In [17]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [18]:
torch.manual_seed(123)
w_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [22]:
print(w_query)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])


In [23]:
print(x_2)

tensor([0.5500, 0.8700, 0.6600])


In [21]:
query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value
print(query_2)

tensor([0.4306, 1.4551])


In [25]:
keys = inputs @ w_key
values = inputs @ w_value
print("Keys shape: ", keys.shape)
print("Value shape: ", values.shape)

Keys shape:  torch.Size([6, 2])
Value shape:  torch.Size([6, 2])


In [26]:
print(keys)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])


In [27]:
print(values)

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])


In [28]:
keys_2 = keys[1]
atten_score_22 = query_2.dot(keys_2)
print(atten_score_22)

tensor(1.8524)


In [30]:
# performing attention scores for all the values
atten_scores_2 = query_2 @ keys.T
print(atten_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [34]:
d_k = keys.shape[-1]
print(d_k)

2


In [37]:
atten_weights_2 = torch.softmax(atten_scores_2 / (d_k**0.5), dim=-1)
print(atten_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [44]:
context_vector_2 = atten_weights_2 @ values
print(context_vector_2)

tensor([0.3061, 0.8210])


### Implementing a self-attention class

In [49]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.w_query = torch.nn.Parameter(torch.rand(d_in, d_out)) 
        self.w_keys = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.w_values = torch.nn.Parameter(torch.rand(d_in, d_out))
        
    def forward(self, x):
        keys = x @ self.w_keys
        values = x @ self.w_values
        queries = x @ w_query
        atten_scores = queries @ keys.T
        atten_weights = torch.softmax(atten_scores / (keys.shape[-1]**0.5), dim=-1)
        context_vector = atten_weights @ values
        return context_vector

In [50]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [53]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.w_key(x)
        values = self.w_value(x)
        query = self.w_query(x)
        attention_scores = query @ keys.T
        attention_weights = torch.softmax(attention_scores / (keys.shape[-1]**0.5), dim=-1)
        context_vector = attention_weights @ values
        return context_vector

In [55]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
